In [ ]:
import rdflib
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF, SDO, RDFS, OWL, DC

import os
from tqdm import tqdm

import sqlparse

In [ ]:
global_graph = Graph()
# https://data.mendeley.com/datasets/zp23s23xpb/1
input_folder = "../dump/soler/"
for filename in tqdm(os.listdir(input_folder)[:1000]):
    f = os.path.join(input_folder, filename)
    try:
        g1 = Graph().parse(f, format='xml')

        #check for literals
        for s, p, o in g1:
            if not rdflib.term._is_valid_uri(o):
                g1.set((s, p, Literal(o)))
             
        global_graph = global_graph + g1
        
    except:
        continue

global_graph.serialize(destination="../datasets/soler.ttl", format="turtle")

In [ ]:
# https://ebiquity.umbc.edu/resource/html/id/82/foafPub-dataset
g = Graph()
g.bind("foaf", FOAF)
g.bind("dc", DC)

with open("dump/foaf_pub/triple_person.sql") as file:
    for i, line in enumerate(tqdm(file)):
        try:
            parsed = sqlparse.parse(line)[0]
            values = str(parsed.tokens[-3])[9:-1].split(',')

            subject = values[5][1:-1]
            predicate = values[4][1:-1]
            object = values[3][1:-1]

            #check all uris
            if not (rdflib.term._is_valid_uri(subject) and rdflib.term._is_valid_uri(predicate)):
                continue

            subject = URIRef(subject)
            predicate = URIRef(predicate)

            # check if literal
            if not rdflib.term._is_valid_uri(object):
                object = Literal(object)
            else:
                object = URIRef(object)
            
            g.add((subject, predicate, object))
        except:
            continue

        #if i > 9000:
        #    break

g.serialize(destination="../datasets/foaf_pub.ttl", format="turtle")

In [ ]:
# FactBench
# https://github.com/DeFacto/FactBench
global_graph = Graph()
input_folder = "dump/FactBench/"
for folder in tqdm(os.listdir(input_folder)):
    folder = os.path.join(input_folder, folder)
    for filename in os.listdir(folder):
        f = os.path.join(folder, filename)
        try:
            g1 = Graph().parse(f, format='ttl')
                
            global_graph = global_graph + g1
            
        except:
            continue

global_graph.serialize(destination="../datasets/factbench.ttl", format="turtle")

In [ ]:
from SPARQLWrapper import SPARQLWrapper, RDFXML
from rdflib import Graph

sparql = SPARQLWrapper("http://dbpedia.org/sparql") # use this endpoint for higher timeout

# https://gist.github.com/tomsaleeba/ff8e145b3efd1127e48baa6512df24e2
sparql.setQuery("""
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
    CONSTRUCT {
        ?actor ?p1 ?o1 .
        ?s2 ?p2 ?actor .
        ?movie ?p3 ?o2 .
        ?s3 ?p4 ?movie .
    }
    WHERE {
        ?actor ?p1 ?o1 {
            SELECT ?actor, ?movie
            WHERE {
                ?movie dbpedia-owl:starring ?actor .
            } 
            order by asc(UCASE(str(?actor)))
            LIMIT 10 # set graph size with this, to high results in partial result only, because of timeout
        } .
        ?s2 ?p2 ?actor FILTER (?p2 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects)) . 

        ?movie ?p3 ?o2 FILTER (?p3 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects)) . 
        ?s3 ?p4 ?movie FILTER (?p4 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects)) . 
    }
""")

sparql.setReturnFormat(RDFXML)
results = sparql.query().convert()
results.serialize(destination="../datasets/dbpedia.ttl", format="turtle")

In [ ]:
# https://gist.github.com/tomsaleeba/ff8e145b3efd1127e48baa6512df24e2

from SPARQLWrapper import SPARQLWrapper, RDFXML, JSON, TURTLE, XML
from rdflib import Graph

sparql = SPARQLWrapper("http://dbpedia.org/sparql") # use this endpoint for higher timeout

sparql.setQuery("""
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
    SELECT ?actor, ?movie
    WHERE {
        ?movie dbpedia-owl:starring ?actor .
    }
    #order by asc(UCASE(str(?actor)))
    LIMIT 100 # set graph size with this
""")

sparql.setReturnFormat(JSON)
actors_movies = sparql.queryAndConvert()
actors_movies

In [ ]:
g = Graph()

for a_m in tqdm(actors_movies["results"]["bindings"]):
    actor = a_m['actor']['value'].rsplit('/', 1)[1]
    sparql.setQuery("""
        PREFIX dbpedia: <http://dbpedia.org/resource/>
        PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
        CONSTRUCT {
            dbpedia:""" + actor + """ ?p1 ?o1 .
            ?s2 ?p2 dbpedia:""" + actor + """ .
        }
        WHERE {
            dbpedia:""" + actor + """ ?p1 ?o1 .
            ?s2 ?p2 dbpedia:""" + actor + """ FILTER (?p2 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects, foaf:primaryTopic)) .
        }
    """)
    sparql.setReturnFormat(TURTLE)
    try:
        results = sparql.query().convert()
        g = g + Graph().parse(data=results, format="turtle")
        g.add((URIRef("http://dbpedia.org/resource/" + actor), RDF.type, URIRef("http://dbpedia.org/ontology/Actor")))
    except:
        continue

unique_movies = set([m['movie']['value'].rsplit('/', 1)[1].replace("(", "\(").replace(")", "\)") for m in actors_movies["results"]["bindings"]])
for movie in tqdm(unique_movies):
    sparql.setQuery("""
        PREFIX dbpedia: <http://dbpedia.org/resource/>
        PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
        CONSTRUCT {
            dbpedia:""" + movie + """ ?p3 ?o2 .
            #?s3 ?p4  dbpedia:""" + movie + """ .
        }
        WHERE {
            dbpedia:""" + movie + """ ?p3 ?o2 FILTER (?p3 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects)) .
            #?s3 ?p4  dbpedia:""" + movie + """ FILTER (?p4 NOT IN (dbo:wikiPageWikiLink, dbo:wikiPageRedirects)) .
        }
    """)

    sparql.setReturnFormat(TURTLE)
    try:
        results = sparql.query().convert()
        g = g + Graph().parse(data=results, format="turtle")
    except:
        continue

g.bind("rdfs", RDFS)
g.add((URIRef("http://dbpedia.org/ontology/starring"), RDFS.domain, URIRef("http://dbpedia.org/ontology/work")))
g.add((URIRef("http://dbpedia.org/ontology/starring"), RDFS.range, URIRef("http://dbpedia.org/ontology/Actor")))
g.serialize(destination="../datasets/dbpedia.ttl", format="turtle")